In [52]:
using ITensors
ITensors.compile()

The directory "C:\Users\jogib\.julia\sysimages" doesn't exist yet, creating it now.

Creating the system image "C:\Users\jogib\.julia\sysimages\sys_itensors.so" containing the compiled version of ITensors. This may take a few minutes.


Precompiling project...
  ✓ NDTensors
  ✓ ITensors
  2 dependencies successfully precompiled in 21 seconds (168 already precompiled)
  2 dependencies precompiled but different versions are currently loaded. Restart julia to access the new versions
┌ Info: ===== Start precompile execution =====
└ @ PackageCompiler C:\Users\jogib\.julia\packages\PackageCompiler\r8Xcy\src\PackageCompiler.jl:245


After sweep 1 energy=-7.266342685676149  maxlinkdim=10 maxerr=2.32E-05 time=27.106
After sweep 2 energy=-7.363892148792675  maxlinkdim=10 maxerr=8.82E-04 time=0.019
After sweep 3 energy=-7.366108714437725  maxlinkdim=10 maxerr=9.18E-04 time=0.028
After sweep 1 energy=-7.299760028906274  maxlinkdim=10 maxerr=2.57E-05 time=18.159
After sweep 2 energy=-7.365738246233662  maxlinkdim=10 maxerr=8.76E-04 time=0.057
After sweep 3 energy=-7.366113596923488  maxlinkdim=10 maxerr=9.22E-04 time=0.032


┌ Info: ===== End precompile execution =====
└ @ PackageCompiler C:\Users\jogib\.julia\packages\PackageCompiler\r8Xcy\src\PackageCompiler.jl:247
┌ Info: PackageCompiler: creating system image object file, this might take a while...
└ @ PackageCompiler C:\Users\jogib\.julia\packages\PackageCompiler\r8Xcy\src\PackageCompiler.jl:366


You will be able to start Julia with a compiled version of ITensors using:
```
~ julia --sysimage C:\Users\jogib\.julia\sysimages\sys_itensors.so
```
and you should see that the startup times and JIT compilation times are substantially improved when you are using ITensors.

In unix, you can create an alias with the Bash command:
```
~ alias julia_itensors="julia --sysimage C:\Users\jogib\.julia\sysimages\sys_itensors.so -e 'using ITensors' -i"
```
which you can put in your `~/.bashrc`, `~/.zshrc`, etc. This also executes `using ITensors` so that ITensors is loaded and ready to use, you can leave off ` -e 'using ITensors' -i` if you don't want that. Then you can start Julia with a version of ITensors installed with the command:
```
~ julia_itensors
```

Note that if you update ITensors to a new version, for example with `using Pkg; Pkg.update("ITensors")`, you will need to run the `ITensors.compile()` command again to recompile the new version of ITensors.



"C:\\Users\\jogib\\.julia\\sysimages\\sys_itensors.so"

In [2]:
let
  i = Index(4,"i")
  j = Index(4,"j")
  k = Index(4,"k")
  l = Index(4,"l")
  T = randomITensor(i,j,k,l)
  U,S,V = svd(T,i,k)   # compute SVD with (i,k) as row indices (indices of U)
  @show hasinds(U,i,k) # = true
  @show hasinds(V,j,l) # = true
  @show T ≈ U*S*V      # = true

  nothing
end

hasinds(U, i, k) = true
hasinds(V, j, l) = true
T ≈ U * S * V = true


In [48]:
using ITensors
let
  # Create 100 spin-one indices
  N = 100
  sites = siteinds("S=1",N)

  # Input operator terms which define
  # a Hamiltonian matrix, and convert
  # these terms to an MPO tensor network
  # (here we make the 1D Heisenberg model)
  ampo = AutoMPO()
  for j=1:N-1
    ampo += "Sz",j,"Sz",j+1
    ampo += 0.5,"S+",j,"S-",j+1
    ampo += 0.5,"S-",j,"S+",j+1
  end
  H = MPO(ampo,sites)

  # Create an initial random matrix product state
  psi0 = randomMPS(sites)

  # Plan to do 5 passes or 'sweeps' of DMRG,
  # setting maximum MPS internal dimensions
  # for each sweep and maximum truncation cutoff
  # used when adapting internal dimensions:
  sweeps = Sweeps(5)
  maxdim!(sweeps, 10,20,100,100,200)
  cutoff!(sweeps, 1E-10)
  @show sweeps

  # Run the DMRG algorithm, returning energy
  # (dominant eigenvalue) and optimized MPS
  energy, psi = dmrg(H,psi0, sweeps)
  println("Final energy = $energy")

  nothing
end

sweeps = Sweeps
1 cutoff=1.0E-10, maxdim=10, mindim=1, noise=0.0E+00
2 cutoff=1.0E-10, maxdim=20, mindim=1, noise=0.0E+00
3 cutoff=1.0E-10, maxdim=100, mindim=1, noise=0.0E+00
4 cutoff=1.0E-10, maxdim=100, mindim=1, noise=0.0E+00
5 cutoff=1.0E-10, maxdim=200, mindim=1, noise=0.0E+00

After sweep 1 energy=-138.061319377806 maxlinkdim=9 time=0.739
After sweep 2 energy=-138.935940264115 maxlinkdim=20 time=2.396
After sweep 3 energy=-138.940082562949 maxlinkdim=91 time=13.322
After sweep 4 energy=-138.940086050552 maxlinkdim=99 time=24.103
After sweep 5 energy=-138.940086063340 maxlinkdim=96 time=24.976
Final energy = -138.94008606334032


In [44]:
?ITensor

search: ITensor ITensors itensor emptyITensor randomITensor diagITensor siteinds



An ITensor is a tensor whose interface is  independent of its memory layout. Therefore it is not necessary to know the ordering of an ITensor's indices, only which indices an ITensor has. Operations like contraction and addition of ITensors automatically handle any memory permutations.

---

```
ITensor(st::TensorStorage, is)
```

Constructor for an ITensor from a TensorStorage and a set of indices. The TensorStorage is copied (the ITensor owns the storage data).

---

```
ITensor(::Tensor)
```

Convert a Tensor to an ITensor with a copy of the storage and indices.

To make an ITensor that shares the same storage as the Tensor, is the function `itensor(::Tensor)`.

---

```
ITensor([::Type{ElT} = Float64, ]inds)
ITensor([::Type{ElT} = Float64, ]inds::Index...)
```

Construct an ITensor filled with zeros having indices `inds` and element type `ElT`. If the element type is not specified, it defaults to `Float64`.

The storage will have `NDTensors.Dense` type.

---

```
ITensor([::Type{ElT} = Float64, ]::UndefInitializer, inds)
ITensor([::Type{ElT} = Float64, ]::UndefInitializer, inds::Index...)
```

Construct an ITensor filled with undefined elements having indices `inds` and element type `ElT`. If the element type is not specified, it defaults to `Float64`.

The storage will have `NDTensors.Dense` type.

---

```
ITensor(x::Number, inds)
ITensor(x::Number, inds::Index...)
```

Construct an ITensor with all elements set to `float(x)` and indices `inds`.

The storage will have `NDTensors.Dense` type.

---

```
ITensor(A::Array, inds)
ITensor(A::Array, inds::Index...)
```

Construct an ITensor from an Array `A` and indices `inds`. The ITensor will be the closest floating point storage to the Array (`float(A)`), and the storage will store a copy of the Array data.

---

```
ITensor([::Type{ElT} = Float64, ][flux::QN = QN(), ]inds)
ITensor([::Type{ElT} = Float64, ][flux::QN = QN(), ]inds::Index...)
```

Construct an ITensor with BlockSparse storage filled with `zero(ElT)` where the nonzero blocks are determined by `flux`.

If `ElT` is not specified it defaults to `Float64`.


In [7]:
N = 100
sites = siteinds("S=1",N)

100-element Vector{Index{Int64}}:
 (dim=3|id=781|"S=1,Site,n=1")
 (dim=3|id=549|"S=1,Site,n=2")
 (dim=3|id=565|"S=1,Site,n=3")
 (dim=3|id=725|"S=1,Site,n=4")
 (dim=3|id=699|"S=1,Site,n=5")
 (dim=3|id=864|"S=1,Site,n=6")
 (dim=3|id=470|"S=1,Site,n=7")
 (dim=3|id=954|"S=1,Site,n=8")
 (dim=3|id=134|"S=1,Site,n=9")
 (dim=3|id=661|"S=1,Site,n=10")
 (dim=3|id=605|"S=1,Site,n=11")
 (dim=3|id=46|"S=1,Site,n=12")
 (dim=3|id=973|"S=1,Site,n=13")
 ⋮
 (dim=3|id=508|"S=1,Site,n=89")
 (dim=3|id=888|"S=1,Site,n=90")
 (dim=3|id=878|"S=1,Site,n=91")
 (dim=3|id=671|"S=1,Site,n=92")
 (dim=3|id=303|"S=1,Site,n=93")
 (dim=3|id=940|"S=1,Site,n=94")
 (dim=3|id=141|"S=1,Site,n=95")
 (dim=3|id=185|"S=1,Site,n=96")
 (dim=3|id=972|"S=1,Site,n=97")
 (dim=3|id=792|"S=1,Site,n=98")
 (dim=3|id=299|"S=1,Site,n=99")
 (dim=3|id=373|"S=1,Site,n=100")

In [8]:
OpName

OpName

In [13]:
ampo = AutoMPO()
for j=1:N-1
  ampo += 0.5,"S+",j,"S-",j+1
  ampo += 0.5,"S-",j,"S+",j+1
  ampo += "Sz",j,"Sz",j+1
end
H = MPO(ampo,sites)

MPO
[1] IndexSet{3} (dim=5|id=668|"Link,l=1") (dim=3|id=781|"S=1,Site,n=1")' (dim=3|id=781|"S=1,Site,n=1") 
[2] IndexSet{4} (dim=5|id=668|"Link,l=1") (dim=5|id=311|"Link,l=2") (dim=3|id=549|"S=1,Site,n=2")' (dim=3|id=549|"S=1,Site,n=2") 
[3] IndexSet{4} (dim=5|id=311|"Link,l=2") (dim=5|id=327|"Link,l=3") (dim=3|id=565|"S=1,Site,n=3")' (dim=3|id=565|"S=1,Site,n=3") 
[4] IndexSet{4} (dim=5|id=327|"Link,l=3") (dim=5|id=324|"Link,l=4") (dim=3|id=725|"S=1,Site,n=4")' (dim=3|id=725|"S=1,Site,n=4") 
[5] IndexSet{4} (dim=5|id=324|"Link,l=4") (dim=5|id=428|"Link,l=5") (dim=3|id=699|"S=1,Site,n=5")' (dim=3|id=699|"S=1,Site,n=5") 
[6] IndexSet{4} (dim=5|id=428|"Link,l=5") (dim=5|id=387|"Link,l=6") (dim=3|id=864|"S=1,Site,n=6")' (dim=3|id=864|"S=1,Site,n=6") 
[7] IndexSet{4} (dim=5|id=387|"Link,l=6") (dim=5|id=284|"Link,l=7") (dim=3|id=470|"S=1,Site,n=7")' (dim=3|id=470|"S=1,Site,n=7") 
[8] IndexSet{4} (dim=5|id=284|"Link,l=7") (dim=5|id=210|"Link,l=8") (dim=3|id=954|"S=1,Site,n=8")' (dim=3|id=954

In [12]:
AutoMPO()

AutoMPO:


In [22]:
print(2*π)

6.283185307179586

In [24]:
⛄=1234

1234

In [25]:
🍆=MPO(ampo,sites)

MPO
[1] IndexSet{3} (dim=5|id=497|"Link,l=1") (dim=3|id=781|"S=1,Site,n=1")' (dim=3|id=781|"S=1,Site,n=1") 
[2] IndexSet{4} (dim=5|id=497|"Link,l=1") (dim=5|id=790|"Link,l=2") (dim=3|id=549|"S=1,Site,n=2")' (dim=3|id=549|"S=1,Site,n=2") 
[3] IndexSet{4} (dim=5|id=790|"Link,l=2") (dim=5|id=526|"Link,l=3") (dim=3|id=565|"S=1,Site,n=3")' (dim=3|id=565|"S=1,Site,n=3") 
[4] IndexSet{4} (dim=5|id=526|"Link,l=3") (dim=5|id=169|"Link,l=4") (dim=3|id=725|"S=1,Site,n=4")' (dim=3|id=725|"S=1,Site,n=4") 
[5] IndexSet{4} (dim=5|id=169|"Link,l=4") (dim=5|id=325|"Link,l=5") (dim=3|id=699|"S=1,Site,n=5")' (dim=3|id=699|"S=1,Site,n=5") 
[6] IndexSet{4} (dim=5|id=325|"Link,l=5") (dim=5|id=160|"Link,l=6") (dim=3|id=864|"S=1,Site,n=6")' (dim=3|id=864|"S=1,Site,n=6") 
[7] IndexSet{4} (dim=5|id=160|"Link,l=6") (dim=5|id=583|"Link,l=7") (dim=3|id=470|"S=1,Site,n=7")' (dim=3|id=470|"S=1,Site,n=7") 
[8] IndexSet{4} (dim=5|id=583|"Link,l=7") (dim=5|id=873|"Link,l=8") (dim=3|id=954|"S=1,Site,n=8")' (dim=3|id=954

In [36]:
b=[1:10;]

10-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [38]:
b
b[end]

10

In [37]:
b[1:3]

3-element Vector{Int64}:
 1
 2
 3

In [56]:

  # Create 100 spin-one indices
  N = 100
  sites = siteinds("Fermion",N)

  # Input operator terms which define
  # a Hamiltonian matrix, and convert
  # these terms to an MPO tensor network
  # (here we make the 1D Heisenberg model)
  ampo = AutoMPO()
  for j=1:N-1
    ampo += "C",j,"Cdag",j+1
    ampo += "Cdag",j,"C",j+1
    
  end
  H = MPO(ampo,sites)

  # Create an initial random matrix product state
  psi0 = randomMPS(sites)

  # Plan to do 5 passes or 'sweeps' of DMRG,
  # setting maximum MPS internal dimensions
  # for each sweep and maximum truncation cutoff
  # used when adapting internal dimensions:
  sweeps = Sweeps(5)
  maxdim!(sweeps, 10,20,100,100,200)
  cutoff!(sweeps, 1E-10)
  @show sweeps

  # Run the DMRG algorithm, returning energy
  # (dominant eigenvalue) and optimized MPS
  energy, psi = dmrg(H,psi0, sweeps)
  println("Final energy = $energy")
  print(psi)


sweeps = Sweeps
1 cutoff=1.0E-10, maxdim=10, mindim=1, noise=0.0E+00
2 cutoff=1.0E-10, maxdim=20, mindim=1, noise=0.0E+00
3 cutoff=1.0E-10, maxdim=100, mindim=1, noise=0.0E+00
4 cutoff=1.0E-10, maxdim=100, mindim=1, noise=0.0E+00
5 cutoff=1.0E-10, maxdim=200, mindim=1, noise=0.0E+00

After sweep 1 energy=-2.752936416690 maxlinkdim=4 time=0.179
After sweep 2 energy=-14.637189392123 maxlinkdim=16 time=0.205
After sweep 3 energy=-28.954097020334 maxlinkdim=64 time=1.317
After sweep 4 energy=-36.135161969615 maxlinkdim=100 time=4.145
After sweep 5 energy=-37.629468472043 maxlinkdim=200 time=16.368
Final energy = -37.629468472043214
MPS
[1] IndexSet{2} (dim=2|id=56|"Link,l=1") (dim=2|id=562|"Fermion,Site,n=1") 
[2] IndexSet{3} (dim=4|id=528|"Link,l=2") (dim=2|id=74|"Fermion,Site,n=2") (dim=2|id=56|"Link,l=1") 
[3] IndexSet{3} (dim=8|id=299|"Link,l=3") (dim=2|id=297|"Fermion,Site,n=3") (dim=4|id=528|"Link,l=2") 
[4] IndexSet{3} (dim=16|id=683|"Link,l=4") (dim=2|id=782|"Fermion,Site,n=4") (di

In [62]:
C=correlation_matrix(psi,"Cdag","C")

LoadError: UndefVarError: correlation_matrix not defined

In [61]:
ITensors.

ITensors

In [71]:
using ITensors
N = 10
s = siteinds()

# Make all of the operators
#X = ops(s, [("X", n) for n in 1:N])

LoadError: MethodError: no method matching siteinds()
[0mClosest candidates are:
[0m  siteinds([91m::MPS[39m) at C:\Users\jogib\.julia\packages\ITensors\cdtHc\src\mps\mps.jl:335
[0m  siteinds([91m::SiteType[39m, [91m::Any[39m; kwargs...) at C:\Users\jogib\.julia\packages\ITensors\cdtHc\src\physics\sitetype.jl:326
[0m  siteinds([91m::String[39m, [91m::Integer[39m; kwargs...) at C:\Users\jogib\.julia\packages\ITensors\cdtHc\src\physics\sitetype.jl:328
[0m  ...

In [129]:
for nm in names(ITensors)
   x = @eval @doc $nm
   print(nm)
end

In [130]:
for i in names(ITensors)
    x=@eval @doc $i
    print(x)
end

In [128]:
using ITensors
let
  # Create 100 spin-one indices
  N = 10
  s = siteinds("S=1/2",N)

# Make all of the operators
  X = op(s, [("x", n) for n in 1:N])
end

LoadError: MethodError: no method matching op(::Vector{Index{Int64}}, ::Vector{Tuple{String, Int64}})
[0mClosest candidates are:
[0m  op(::Vector{var"#s199"} where var"#s199"<:Index, [91m::AbstractString[39m, [91m::Int64...[39m; kwargs...) at C:\Users\jogib\.julia\packages\ITensors\cdtHc\src\physics\sitetype.jl:249

In [118]:
N = 10
  s = siteinds("S=1/2",N)

# Make all of the operators
  X = ops(s, [("x", n) for n in 1:N])

LoadError: UndefVarError: ops not defined

In [131]:
OpSum()

LoadError: UndefVarError: OpSum not defined